In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from common.models import simple_conv


c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [76]:
bs = 4
tl = 1024
in_feats = 460
base_width = 512
down_layers = 3

encoder = simple_conv.Encoder(in_feats, down_layers, base_width, [False, True, True])

model = simple_conv.SimpleSegConv(encoder, 200)

In [77]:
ins = torch.randn(bs, tl, in_feats)
ins.size()

torch.Size([4, 1024, 460])

In [78]:
x = model(ins)

torch.Size([4, 460, 1024])
torch.Size([4, 2048, 512])
torch.Size([4, 1024, 1024])
torch.Size([4, 200, 1024])


In [79]:
model.encoder.reduction

1

In [73]:
x.size()

torch.Size([4, 1024, 200])

In [74]:
model

SimpleSegConv(
  (encoder): Encoder(
    (0): Sequential(
      (0): Conv1d(460, 512, kernel_size=(3,), stride=(2,), padding=(1,), bias=False, padding_mode=reflect)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv1d(512, 1024, kernel_size=(3,), stride=(2,), padding=(1,), bias=False, padding_mode=reflect)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv1d(1024, 2048, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=reflect)
      (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (blocks): ModuleList(
    (0): Sequential(
      (0): Conv1d(2048, 1024, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=reflect)
      (1): BatchNorm1d(1024, eps